In [1]:
# Add confidence interval
import config_cat_embedding
import pandas as pd
import numpy as np
import random
import time
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
# from sklearn.svm import SVC
from xgboost import XGBClassifier
from scikeras.wrappers import KerasClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, roc_auc_score)
from scipy import stats

from data_prep import bank_data_prep, adult_data_prep
from embedding_helper import create_network

# Set the random seed for reproducibility
random.seed(42)
np.random.seed(42)

# Load and preprocess data
data_path = config_cat_embedding.paths['data']
bank_data = pd.read_csv(data_path + 'bank-additional-full.csv', sep=';')

df_bank, cat_cols = bank_data_prep(bank_data)

X = df_bank.iloc[:, :-1]
y = df_bank.y

# Define the classifiers
seed = 42
# We will determine the number_of_features inside the cross-validation loop after preprocessing

models = [
    ('LR', LogisticRegression(solver='lbfgs', random_state=seed, max_iter=1000)),
    ('DT', DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=seed)),
    ('RF', RandomForestClassifier(n_estimators=200, max_depth=5, random_state=seed, min_samples_leaf=3)),
    ('KNN', KNeighborsClassifier(n_neighbors=3)),
    ('XGB', XGBClassifier(eval_metric='logloss', random_state=seed)),
    # ('SVM', SVC(gamma='scale', random_state=seed, probability=True)),
    ('MLP', KerasClassifier(
        model=create_network,
        epochs=100, batch_size=100, verbose=0, random_state=seed))
]

# Cross-validation setup
cv = StratifiedKFold(n_splits=20, shuffle=True, random_state=seed)

# Function to calculate confidence intervals
def confidence_interval(data, confidence=0.95):
    n = len(data)
    m = np.mean(data)
    std_err = stats.sem(data)
    h = std_err * stats.t.ppf((1 + confidence) / 2, n - 1)
    return m, m - h, m + h

# Loop over models
for name, model in models:
    print(f"Classifier: {name}")
    # Lists to store metrics for each fold
    accuracies = []
    precisions = []
    recalls = []
    f1s = []
    roc_aucs = []
    
    # Start the timer before cross-validation
    start_time = time.time()
    
    fold = 1
    for train_index, test_index in cv.split(X, y):
        # Split the data into training and testing sets for this fold
        X_train_fold, X_test_fold = X.iloc[train_index].copy(), X.iloc[test_index].copy()
        y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]
        
        # Preprocess data within the fold
        # WOE encoding
        import category_encoders as ce
        woe_encoder = ce.WOEEncoder(cols=cat_cols)
        X_train_enc = woe_encoder.fit_transform(X_train_fold, y_train_fold)
        X_test_enc = woe_encoder.transform(X_test_fold)
        
        # Standard scaling
        stc = StandardScaler()
        X_train_scaled = stc.fit_transform(X_train_enc)
        X_test_scaled = stc.transform(X_test_enc)
        
        # Update number_of_features for MLP
        number_of_features = X_train_scaled.shape[1]
        if name == 'MLP':
            # Update the model with the correct number of features
            model.set_params(model__number_of_features=number_of_features)
        
        # Fit the model
        model.fit(X_train_scaled, y_train_fold)
        # Predict on the test fold
        y_pred_fold = model.predict(X_test_scaled)
        
        # Get prediction probabilities for ROC AUC
        if hasattr(model, "predict_proba"):
            y_pred_prob_fold = model.predict_proba(X_test_scaled)[:, 1]
        else:
            # For classifiers without predict_proba, use decision_function
            y_pred_prob_fold = model.decision_function(X_test_scaled)
            # Normalize the decision function output to [0,1] range
            y_pred_prob_fold = (y_pred_prob_fold - y_pred_prob_fold.min()) / (y_pred_prob_fold.max() - y_pred_prob_fold.min())
        
        # Calculate metrics
        accuracies.append(accuracy_score(y_test_fold, y_pred_fold))
        precisions.append(precision_score(y_test_fold, y_pred_fold, zero_division=0))
        recalls.append(recall_score(y_test_fold, y_pred_fold))
        f1s.append(f1_score(y_test_fold, y_pred_fold))
        roc_aucs.append(roc_auc_score(y_test_fold, y_pred_prob_fold))
        
        # print(f"Fold {fold} completed.")
        fold += 1
    
    # Stop the timer after cross-validation
    end_time = time.time()
    total_computation_time = end_time - start_time  # Total time for the model
    
    # Calculate mean and confidence intervals
    acc_mean, acc_ci_lower, acc_ci_upper = confidence_interval(accuracies)
    prec_mean, prec_ci_lower, prec_ci_upper = confidence_interval(precisions)
    rec_mean, rec_ci_lower, rec_ci_upper = confidence_interval(recalls)
    f1_mean, f1_ci_lower, f1_ci_upper = confidence_interval(f1s)
    roc_mean, roc_ci_lower, roc_ci_upper = confidence_interval(roc_aucs)
    
    # Print results
    print(f"Accuracy: {acc_mean:.3f} (95% CI: {acc_ci_lower:.3f} - {acc_ci_upper:.3f})")
    print(f"Precision: {prec_mean:.3f} (95% CI: {prec_ci_lower:.3f} - {prec_ci_upper:.3f})")
    print(f"Recall: {rec_mean:.3f} (95% CI: {rec_ci_lower:.3f} - {rec_ci_upper:.3f})")
    print(f"F1 Score: {f1_mean:.3f} (95% CI: {f1_ci_lower:.3f} - {f1_ci_upper:.3f})")
    print(f"ROC AUC: {roc_mean:.3f} (95% CI: {roc_ci_lower:.3f} - {roc_ci_upper:.3f})")
    print(f"Total Computation Time: {total_computation_time:.3f} seconds\n")


Classifier: LR
Accuracy: 0.906 (95% CI: 0.904 - 0.908)
Precision: 0.655 (95% CI: 0.639 - 0.671)
Recall: 0.346 (95% CI: 0.330 - 0.362)
F1 Score: 0.452 (95% CI: 0.437 - 0.468)
ROC AUC: 0.913 (95% CI: 0.909 - 0.917)
Total Computation Time: 10.442 seconds

Classifier: DT
Accuracy: 0.906 (95% CI: 0.904 - 0.908)
Precision: 0.650 (95% CI: 0.632 - 0.668)
Recall: 0.358 (95% CI: 0.342 - 0.374)
F1 Score: 0.461 (95% CI: 0.445 - 0.478)
ROC AUC: 0.850 (95% CI: 0.844 - 0.857)
Total Computation Time: 11.998 seconds

Classifier: RF
Accuracy: 0.902 (95% CI: 0.900 - 0.903)
Precision: 0.755 (95% CI: 0.730 - 0.780)
Recall: 0.186 (95% CI: 0.170 - 0.202)
F1 Score: 0.298 (95% CI: 0.276 - 0.319)
ROC AUC: 0.912 (95% CI: 0.907 - 0.917)
Total Computation Time: 85.355 seconds

Classifier: KNN
Accuracy: 0.892 (95% CI: 0.890 - 0.895)
Precision: 0.530 (95% CI: 0.514 - 0.546)
Recall: 0.396 (95% CI: 0.378 - 0.414)
F1 Score: 0.453 (95% CI: 0.436 - 0.469)
ROC AUC: 0.790 (95% CI: 0.781 - 0.799)
Total Computation Time: 62.

In [2]:
import config_cat_embedding
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
from tqdm import tqdm_notebook as tqdm

from data_prep import bank_data_prep, adult_data_prep
from embedding_helper import create_network
import time
import random

# Set the random seed for reproducibility
random.seed(42)
np.random.seed(42)

data_path = config_cat_embedding.paths['data']
data_path_out = config_cat_embedding.paths['data_output']
bank_data = pd.read_csv(data_path + 'bank-additional-full.csv', sep=';')

df_bank, cat_cols = bank_data_prep(bank_data)

X = df_bank.iloc[:, :-1]
target = df_bank.y

X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.2, random_state=1500)

# One-hot encoding
import category_encoders as ce
woe_encoder = ce.WOEEncoder(cols=cat_cols)
X_train_ohe = woe_encoder.fit_transform(X_train, y_train)
X_test_ohe = woe_encoder.transform(X_test)  # Use transform() instead of fit_transform()

# Standard scaling
stc = StandardScaler()
X_train_scaled = stc.fit_transform(X_train_ohe)
X_test_scaled = stc.transform(X_test_ohe)

number_of_features = X_train_scaled.shape[1]  # Number of features in the input data

# Define the classifiers
seed = 42
models = [
    ('LR', LogisticRegression(solver='lbfgs', random_state=seed, max_iter=1000)),
    ('DT', DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=seed)),
    ('RF', RandomForestClassifier(n_estimators=200, max_depth=5, random_state=seed, min_samples_leaf=3)),
    ('KNN', KNeighborsClassifier(n_neighbors=3)),
    ('XGB', XGBClassifier(eval_metric='logloss', use_label_encoder=False)),
    ('SVM', SVC(gamma='scale', random_state=seed, probability=True)),
    ('MLP', KerasClassifier(build_fn=lambda: create_network(number_of_features), epochs=100, batch_size=100, verbose=0))
]

# Train and evaluate the models
for name, model in models:
    start_time = time.time()  # Start time
    
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    y_pred_prob = model.predict_proba(X_test_scaled)
    
    end_time = time.time()  # End time
    computation_time = end_time - start_time  # Computation time
    
    print(f"Classifier: {name}")
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred, digits=3))  # Set digits to 3 for three decimal places
    print(f"ROC AUC Score: {roc_auc_score(y_test, y_pred_prob[:, 1]):.3f}")
    print(f"Computation Time: {computation_time:.3f} seconds")
    print()

C:\Users\xg16060\Miniconda3\envs\survey_paper\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
C:\Users\xg16060\AppData\Local\Temp\1\ipykernel_2396\3485041637.py:58: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  ('MLP', KerasClassifier(build_fn=lambda: create_network(number_of_features), epochs=100, batch_size=100, verbose=0))


Classifier: LR
[[7133  161]
 [ 626  318]]
              precision    recall  f1-score   support

           0      0.919     0.978     0.948      7294
           1      0.664     0.337     0.447       944

    accuracy                          0.904      8238
   macro avg      0.792     0.657     0.697      8238
weighted avg      0.890     0.904     0.890      8238

ROC AUC Score: 0.916
Computation Time: 0.033 seconds

Classifier: DT
[[7106  188]
 [ 618  326]]
              precision    recall  f1-score   support

           0      0.920     0.974     0.946      7294
           1      0.634     0.345     0.447       944

    accuracy                          0.902      8238
   macro avg      0.777     0.660     0.697      8238
weighted avg      0.887     0.902     0.889      8238

ROC AUC Score: 0.848
Computation Time: 0.024 seconds

Classifier: RF
[[7237   57]
 [ 755  189]]
              precision    recall  f1-score   support

           0      0.906     0.992     0.947      7294
   